In [72]:
import pandas as pd
import numpy as np
import os
import random
import hashlib

data_path = '../../analysis/pre'

def site_id_hash(site_id):
    sha256_hash = hashlib.sha256(site_id.encode()).hexdigest()
    return sha256_hash[:7]

In [67]:
# Get all data of alternative sites

site_info_alt = pd.read_csv(data_path + '/data-fetching/alternative-sites/site_info_20230101.csv')
weather_data_alt = pd.read_csv(data_path + '/data-fetching/alternative-sites/df_weather-20230101-20240501.csv')

weather_data_alt.rename(columns={'ts': 'time'}, inplace=True)
weather_data_alt['time'] = pd.to_datetime(weather_data_alt['time'])

power_data_alt = pd.DataFrame()

temp_site_info = []

for index, row in site_info_alt.iterrows():
    site = row
    cluster = 0 # cluster 0 is chosen (= no clustering)
    site_id = site['site_id']
    new_entry = {
        'site_id': site_id, 
        'cluster': cluster,
        'lat': 0, # site['lat'],
        'lng': 0, # site['lng'],
        'zip': site['zip'],
        'country': site['country'],
        'kwp': site['kwp'],
    }
    temp_site_info.append(new_entry)
    site_power_data = pd.read_csv(data_path + f'/data-fetching/alternative-sites/df_{site_id}-20230101-20240501.csv')
    power_data_alt = pd.concat([power_data_alt, site_power_data])

site_info_alt = pd.DataFrame(temp_site_info)
power_data_alt['time'] = pd.to_datetime(power_data_alt['time'])

print('Alternative sites data loaded')
print(f'Power data length: {len(power_data_alt)}')


Alternative sites data loaded
Power data length: 1277529


In [68]:
# Get all data of main sites

site_info = pd.read_csv(data_path + '/data-fetching/site_info_20230101.csv')
weather_data = pd.read_csv(data_path + '/data-fetching/df_weather-20230101-20240101.csv')
clusters = pd.read_csv(data_path + '/site-clustering//closest_sites.csv')

weather_data.rename(columns={'ts': 'time'}, inplace=True)
weather_data['time'] = pd.to_datetime(weather_data['time'])

power_data = pd.DataFrame()

temp_site_info = []

for index, row in site_info.iterrows():
    site = row
    cluster = clusters[clusters['site_id'] == site['site_id']].iloc[0]
    site_id = site['site_id']
    new_entry = {
        'site_id': site_id, 
        'cluster': cluster['cluster'],
        'lat': 0, # site['lat'],
        'lng': 0, # site['lng'],
        'zip': site['zip'],
        'country': site['country'],
        'kwp': site['kwp'],
    }
    temp_site_info.append(new_entry)
    site_power_data = pd.read_csv(data_path + f'/data-fetching/df_{site_id}-20230101-20240101.csv')
    power_data = pd.concat([power_data, site_power_data])

site_info = pd.DataFrame(temp_site_info)
power_data['time'] = pd.to_datetime(power_data['time'])

print('Main sites data loaded')
print(f'Power data length: {len(power_data)}')

Main sites data loaded
Power data length: 878496


In [69]:
# concat alternative and main sites
site_info = pd.concat([site_info, site_info_alt], ignore_index=True)
power_data = pd.concat([power_data, power_data_alt], ignore_index=True)
weather_data = pd.concat([weather_data, weather_data_alt], ignore_index=True)

In [70]:

print('All data loaded')
print(f'Site info length: {len(site_info)}')
print(f'Power data length: {len(power_data)}')
print(f'Weather data length: {len(weather_data)}')

print(site_info)    


All data loaded
Site info length: 30
Power data length: 2156025
Weather data length: 239526
                                 site_id  cluster  lat  lng    zip country  \
0   0106fa6a-06d7-4991-a254-93d4ee1464a3        0    0    0   4073      AT   
1   1df9e630-c947-4b9a-89e2-c77b643506c5        1    0    0   1220      AT   
2   2bbfa73e-2a2e-473b-8d1c-f6b87bff8403        7    0    0   8055      AT   
3   50eb6cac-7dbb-44ae-83c6-9b4879a57425        7    0    0   8401      AT   
4   6045bd8d-1754-4963-b3cb-c703110ac5ac        0    0    0   4100      AT   
5   60f50416-f2b0-4191-b82d-c71ba439d558        7    0    0   8041      AT   
6   70118ad9-4de9-473e-8415-27e5f641a7a1        1    0    0   1220      AT   
7   98e065ed-de0e-4973-8e43-7020defe23b0        1    0    0   1220      AT   
8   a62e58d7-8b9a-4531-9ca5-323d136dcec0        0    0    0   4073      AT   
9   a7b10e3f-e8a6-4f58-affd-c50b5f2c4201        7    0    0   8054      AT   
10  aa13d3f3-04b2-43d1-b9ba-16adf87a4bc5        0 

In [71]:
os.makedirs(f'../test_data', exist_ok=True)

df_all = pd.DataFrame()

for site_id in site_info['site_id']:
    print("Site ID:", site_id)
    site = site_info[site_info['site_id'] == site_id].iloc[0]
    power_data_site = power_data[(power_data['site_id'] == site_id)]
    weather_data_site = weather_data[(weather_data['zip'] == site['zip'])]
    
    power_data_site['time'] = pd.to_datetime(power_data_site['time'])
    weather_data_site['time'] = pd.to_datetime(weather_data_site['time'])

    power_data_site = power_data_site.sort_values('time')


    if len(weather_data_site) != 0:
        weather_data_site = weather_data_site.sort_values('time')

        duplicated_weather = []

        # Duplicate weather data to match the power data
        for index, row in weather_data_site.iterrows():
            # Convert timestamp to datetime object
            ts = row['time']
            # Append the original row to the duplicated list
            duplicated_weather.append(row)
            # Duplicate the row for three additional times (15-minute intervals)
            for _ in range(3):
                ts += pd.Timedelta(minutes=15)
                # Create a copy of the row and update the timestamp
                new_row = row.copy()
                new_row['time'] = ts
                # Append the duplicated row to the list
                duplicated_weather.append(new_row)
        
        generated_weather = pd.DataFrame(duplicated_weather)
        generated_weather['site_id'] = site_id
        power_data_site['site_id'] = site_id

        df = pd.merge(generated_weather, power_data_site, on=['site_id', 'time'])

        df = df[['time', 'site_id', 'value_key', 'avg', 'solar_rad', 'temp', 'precip', 'rh', 'ghi', 'snow_depth']]


        weather_columns = ['solar_rad', 'temp', 'precip', 'rh', 'ghi', 'snow_depth']
        for row in df.iterrows():
            # get row 1h and 2h before
            row_1h = df[(df['time'] == row[1]['time'] - pd.Timedelta(hours=1)) & (df['value_key'] == row[1]['value_key'])]
            row_2h = df[(df['time'] == row[1]['time'] - pd.Timedelta(hours=1)) & (df['value_key'] == row[1]['value_key'])]
            
            # set values if data is available
            if len(row_1h) != 0:
                for column in weather_columns:
                    df.loc[row[0], f'{column}_1h'] = row_1h[column].values[0]
            else:
                for column in weather_columns:
                    df.loc[row[0], f'{column}_1h'] = np.nan

            if len(row_2h) != 0:
                for column in weather_columns:
                    df.loc[row[0], f'{column}_2h'] = row_2h[column].values[0]
            else:
                for column in weather_columns:
                    df.loc[row[0], f'{column}_2h'] = np.nan

        site['weather_data'] = True
    else:
        df = df[['time', 'site_id', 'value_key', 'avg']]
        site['weather_data'] = False
    

    df.sort_values('time', inplace=True)

    for row in df.iterrows():
        row_24h = df[(df['time'] == row[1]['time'] - pd.Timedelta(hours=24)) & (df['value_key'] == row[1]['value_key'])]

        if len(row_24h) != 0:
            df.loc[row[0], 'avg_24h'] = row_24h['avg'].values[0]
        else:
            df.loc[row[0], 'avg_24h'] = np.nan


    os.makedirs(f'../data/test_data/{site_id}', exist_ok=True)
    distinct_value_keys = df['value_key'].unique()
    for value_key in distinct_value_keys:
        filtered_df = df[df['value_key'] == value_key]
        filtered_df.to_csv(f'../data/test_data/{site_id}/{value_key}.csv', index=False)

    df_all = pd.concat([df_all, df], ignore_index=True)
    site_info.loc[site_info['site_id'] == site_id, 'weather_data'] = site['weather_data']

    # create directories recursively
    os.makedirs(f'../data/test_data/{site_id}', exist_ok=True)

    site.to_csv(f'../data/test_data/{site_id}/site_info.csv', index=True)

site_info.to_csv(f'../data/test_data/site_info.csv', index=False)

Site ID: 0106fa6a-06d7-4991-a254-93d4ee1464a3


/var/folders/35/hfsjshqn6zxgxjhjt_p2s4hw0000gp/T/ipykernel_1438/724645186.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  power_data_site['time'] = pd.to_datetime(power_data_site['time'])
/var/folders/35/hfsjshqn6zxgxjhjt_p2s4hw0000gp/T/ipykernel_1438/724645186.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data_site['time'] = pd.to_datetime(weather_data_site['time'])


KeyboardInterrupt: 

In [151]:
window_size = 7  # The size of each window (in days)

def are_subsequent(dates):
    """Check if dates are subsequent."""
    return np.all(np.diff(dates) == pd.Timedelta(days=1))

for site_id in site_info['site_id']:
    site = site_info[site_info['site_id'] == site_id].iloc[0]
    site_power_data = power_data[(power_data['site_id'] == site_id)]

    unique_days = np.sort(site_power_data['time'].dt.date.unique())

    # Split the unique days into windows
    windows = [unique_days[i:i+window_size] for i in range(0, len(unique_days), window_size)]

    # Filter out non-subsequent windows
    complete_windows = [window for window in windows if len(window) == window_size and are_subsequent(window)]

    # Shuffle the windows
    random.shuffle(complete_windows)

    # Select a fixed number of windows for testing
    test_windows_size = 4  # Change this to the number of windows you want for testing
    test_windows, training_windows = complete_windows[:test_windows_size], complete_windows[test_windows_size:]

    # Sort the windows
    test_windows.sort(key=lambda x: x[0])
    training_windows.sort(key=lambda x: x[0])

    # Convert windows into DataFrame
    training_windows_df = pd.DataFrame(training_windows)
    test_windows_df = pd.DataFrame(test_windows)

    # Write windows to CSV files
    training_windows_df.to_csv(f'../data/test_data/{site_id}/training_windows.csv', index=False, header=True)
    test_windows_df.to_csv(f'../data/test_data/{site_id}/test_windows.csv', index=False, header=True)

In [109]:
# print max values for solar_rad and ghi

print("Max solar_rad:", df_all['solar_rad'].max())
print("Max ghi:", df_all['ghi'].max())
print("Max temp:", df_all['temp'].max())
print("Max precip:", df_all['precip'].max())
print("Max snow depth:", df_all['snow_depth'].max())

# min values
print("Min solar_rad:", df_all['solar_rad'].min())
print("Min ghi:", df_all['ghi'].min())



Max solar_rad: 956.2
Max ghi: 956.21
Max temp: 35.1
Max precip: 14.78125
Max snow depth: 1178.6
Min solar_rad: 0.0
Min ghi: 0.0
